In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize between [-1, 1]
])

In [5]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16020227.81it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 467166.07it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4388308.87it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2737144.94it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
#Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 28 * 28),
            nn.Tanh()  # Output scaled between [-1, 1]
        )
    
    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), 1, 28, 28)  # Reshape to image dimensions
        return x

In [7]:
#Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Output between [0, 1]
        )
    
    def forward(self, x):
        x = x.view(x.size(0), 28 * 28)  # Flatten the input
        x = self.model(x)
        return x

In [8]:
# Initialize the Generator and Discriminator
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

# Loss function (Binary Cross Entropy Loss)
criterion = nn.BCELoss()

In [9]:
# Number of epochs and latent vector size
epochs = 50
latent_dim = 100
fixed_noise = torch.randn(64, latent_dim).to(device)  # For generating samples during training

# Training Loop
for epoch in range(epochs):
    for batch_idx, (real_imgs, _) in enumerate(train_loader):
        # Training Discriminator
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0)
        real_labels = torch.ones(batch_size, 1).to(device)  # Label 1 for real images
        fake_labels = torch.zeros(batch_size, 1).to(device)  # Label 0 for fake images
        
        # Real images
        outputs = discriminator(real_imgs)
        d_loss_real = criterion(outputs, real_labels)
        
        # Fake images
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = generator(z)
        outputs = discriminator(fake_imgs.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        
        # Total discriminator loss
        d_loss = d_loss_real + d_loss_fake
        
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()
        
        # Training Generator
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = generator(z)
        outputs = discriminator(fake_imgs)
        g_loss = criterion(outputs, real_labels)  # Try to fool the discriminator
        
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")
    
    # Save generated samples every epoch
    if (epoch+1) % 10 == 0:
        save_image(fake_imgs.data[:25], f"mnist_generated_{epoch+1}.png", nrow=5, normalize=True)

Epoch [1/50] | D Loss: 0.3784 | G Loss: 3.7614
Epoch [2/50] | D Loss: 0.6523 | G Loss: 1.6349
Epoch [3/50] | D Loss: 0.4870 | G Loss: 2.9242
Epoch [4/50] | D Loss: 0.4322 | G Loss: 4.0320
Epoch [5/50] | D Loss: 0.3495 | G Loss: 3.1899
Epoch [6/50] | D Loss: 0.3851 | G Loss: 2.3943
Epoch [7/50] | D Loss: 0.5256 | G Loss: 3.4424
Epoch [8/50] | D Loss: 0.5313 | G Loss: 2.3141
Epoch [9/50] | D Loss: 0.2960 | G Loss: 2.2898
Epoch [10/50] | D Loss: 0.4419 | G Loss: 2.5318
Epoch [11/50] | D Loss: 0.3611 | G Loss: 2.9232
Epoch [12/50] | D Loss: 0.4444 | G Loss: 2.5243
Epoch [13/50] | D Loss: 0.3998 | G Loss: 2.3706
Epoch [14/50] | D Loss: 0.7718 | G Loss: 2.1743
Epoch [15/50] | D Loss: 0.7300 | G Loss: 2.1707
Epoch [16/50] | D Loss: 0.7669 | G Loss: 1.4152
Epoch [17/50] | D Loss: 0.8957 | G Loss: 1.8828
Epoch [18/50] | D Loss: 0.8579 | G Loss: 1.6284
Epoch [19/50] | D Loss: 1.1064 | G Loss: 1.4009
Epoch [20/50] | D Loss: 0.7030 | G Loss: 1.6915
Epoch [21/50] | D Loss: 0.9767 | G Loss: 1.9674
E

In [10]:
# Generate and save new images after training
z = torch.randn(64, latent_dim).to(device)
generated_imgs = generator(z)
save_image(generated_imgs.data[:25], "mnist_final_generated.png", nrow=5, normalize=True)